In [1]:
import os
import numpy as np
import pandas as pd
import scipy.stats
from sklearn.linear_model import LinearRegression as lr
import datetime as dt
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
os.chdir('C:\\Users\\VanBuren\\Desktop\\Coursework\\Capstone\\Coding_analysis\\Test_Data')

In [9]:
#NYDEC CALIBRATION SCHEME
#----USEFUL FOR NON-SUBWAY CALIBRATIONS: INDOOR/OUTDOOR TIME.
#designate timespan we want
start = dt.datetime.strptime('2020-07-10', '%Y-%m-%d')
end = dt.datetime.strptime('2021-01-27', '%Y-%m-%d')


# Function to remove timezone awareness from utc datetime fields (for easier manipulation),
#     and to trim data to start and end timestamps:
def utc_delocalize(frame, field, start, end):
    frame[field] = pd.to_datetime(frame[field]).dt.tz_localize(None)
    frame = frame.loc[(frame[field] >= start) & ((frame[field] < end))]
    frame = frame.set_index(frame[field])
    return frame


#Read in the NY DEC data, combine date/time columns into one datetime field for easier analysis
refdata = pd.read_csv('nydec.csv',header = 1,parse_dates=[['Date','Time']])
refdata = refdata.rename(columns={"PM25C_ug/m3LC":"PM2.5_ref"})


# Localize refdata to UTC (then remove timezone awareness)
#     refdata's datetime field was captured in Eastern time, but the field is not timezone-aware.
#         First set timezone awareness, then convert to UTC:
refdata['Date_Time'] = refdata['Date_Time'].dt.tz_localize('US/Eastern', ambiguous = 'NaT')
refdata['Date_Time'] = refdata['Date_Time'].dt.tz_convert('UTC')
refdata = utc_delocalize(refdata, 'Date_Time', start, end)


# read in purpleair datasets: hour averages (for best comparison to refdata data)
#     then limit range of PA sensor data to desired timespan
#     then set index to timestamp
sensA = pd.read_csv('school_4_A_hr.csv',header = 0)
sensA = sensA.rename(columns={"Temperature_F":"Temp","PM2.5_ATM_ug/m3":"PM2.5A"})
sensA = utc_delocalize(sensA, 'created_at', start, end)


sensB = pd.read_csv('school_4_B_hr.csv',header = 0)
sensB = sensB.rename(columns={"PM2.5_ATM_ug/m3":"PM2.5B"})
sensB = utc_delocalize(sensB, 'created_at', start, end)


#Get all info onto a single dataframe
df = pd.concat([sensA[['Temp','Humidity_%','PM2.5A']], sensB['PM2.5B'], refdata['PM2.5_ref']], axis = 1)
df['PM2.5_test'] = df[['PM2.5A','PM2.5B']].mean(axis = 1)


#Null values will cause problems in statswork later on, so eliminate them now.
df_nulls = ['PM2.5_ref', 'PM2.5_test', 'Temp', 'Humidity_%']

def null_remover(frame, fields):
    for field in fields:
        frame = frame.loc[frame[field].notnull()]
    return frame

df = null_remover(df, df_nulls)
#------------------------------------------------------------------------------------------------------------


#statswork for multiple regression
#Parameters for regression in X, assume they're predictors for nydec data as y

#trained model: nydec = x0 + x1[temp] + x2[humidity] + x3[PA data]

X = np.array(df[['Temp','Humidity_%','PM2.5_test']])
y = np.array(df['PM2.5_ref'])
reg = lr(fit_intercept = False).fit(X,y)

#-------------------------------------------
# CALIBRATION FUNCTION FOR OUTDOOR/INDOOR (NON-TRAIN/PLATFORM) DATA
def reg_out(X):
    return np.dot(X,reg.coef_)
#-------------------------------------------

#R-square:
print(reg.score(X,y))

#run that model on the parameters
df['PM2.5_calib'] = np.dot(X,reg.coef_)

0.5328656567210689


In [7]:
#CALIBRATION SCHEME B
#Luglio curve--meant for on the train platforms

# Based on mass calculations from air filter collections
filt = pd.read_csv('Gravcal.csv', header = 0)


X = np.column_stack([np.zeros(len(filt['PA PM2.5 avg'])),filt['PA PM2.5 avg']])
y = np.array(filt['Grav(average)'])
calibs = lr(fit_intercept = False).fit(X,y)


# -------------------------------------------------------
# CALIBRATION FOR UNDERGROUND PLATFORM & TRAIN DATA
def reg_plat(X):
    return calibs.coef_[1] * X
# -------------------------------------------------------

In [8]:
# Calibration Scheme C
# For dealing with PA and aboveground trains:
#!!! AND HERE, GET RID OF THE LAMBDA
reg_sir = lambda q: q*(30.9/36.2)

# -------------------------------------------------------
# CALIBRATION FOR ABOVEGROUND PLATFORMS AND TRAINS
def reg_sir(X):
    return X * (30.9/36.2)
# -------------------------------------------------------

In [5]:
# Comprehensive calibration function to calibrate datasets according to location code

def calibrate(frame):
    
    #Establish which code gets calibrated how
    codes_nydec = ['O','I','F']
    codes_luglio = ['PU','T','E']
    codes_sir = ['PA']
    
    
    #then calibrate based on Loc_code:
    frame['PM2.5_calib'] = np.nan
    
    #nydec calibration
    X_nydec = frame[['Temp','Humidity_%','PM2.5']].loc[frame['Loc_code'].isin(codes_nydec)]
    frame['PM2.5_calib'].loc[frame['Loc_code'].isin(codes_nydec)] = reg_out(X_nydec)
    
    #luglio calibration
    X_luglio = frame['PM2.5'].loc[frame['Loc_code'].isin(codes_luglio)]
    frame['PM2.5_calib'].loc[frame['Loc_code'].isin(codes_luglio)] = reg_plat(X_luglio)
    
    #SIR calibration for aboveground plats
    X_sir = frame['PM2.5'].loc[frame['Loc_code'].isin(codes_sir)]
    frame['PM2.5_calib'].loc[frame['Loc_code'].isin(codes_sir)] = reg_sir(X_sir)
    
    #Special aboveground train calibration using SIR
    X_sir2 = frame['PM2.5'].loc[(frame['Loc_code']=='T') & (frame['Outside? (Y/N)'] == 'Y')]
    frame['PM2.5_calib'].loc[(frame['Loc_code']=='T') & (frame['Outside? (Y/N)'] == 'Y')] = reg_sir(X_sir2)
    
    
    return frame

In [8]:
# Keeping the other imports below for posterity and/or potential emergencies? idk, just don't wanna delete yet
dfsub = pd.read_csv('Master_compiled_BB.csv',header = 0, parse_dates = [['Date', 'Time_NYC']]).rename(
    columns = {'Temperature (F)': 'Temp','Humidity (%)':'Humidity_%','PA PM2.5 (um/m3)':'PM2.5'})

dfsub = dfsub.loc[dfsub['PM2.5'].notnull() & dfsub['Temp'].notnull() & dfsub['Humidity_%'].notnull() & 
                  dfsub['Location'].notnull()]
dfsub = calibrate(dfsub)
# dfsub.loc[(dfsub['Loc_code']=='T') & (dfsub['Outside? (Y/N)'] == 'Y')]
dfsub.to_csv('Masterlist.csv')

C:\Users\VanBuren\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
